<a href="https://colab.research.google.com/github/mpekar1/DSS-IDI-modelling/blob/main/Markov_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markov Chain Modelling

**Objective:**
Analyse trends among individuals ceasing to receive DSS services and model the transition probabilities across different states influenced by various factors, including age transitions around 18-20 years.

## Resources

Emmanuel F. Drabo and William V. Padula, Introduction to Markov modeling In: Handbook of Applied Health Economics in
Vaccines. Edited by: David Bishai, Logan Brenzel and William V. Padula, Oxford University Press. © Oxford University Press 2023.
DOI: 10.1093/oso/9780192896087.003.0022
https://academic.oup.com/book/45888/chapter/401430223


http://cran.nexr.com/web/packages/markovchain/vignettes/markovchainCrashIntro.pdf

https://www.datacamp.com/tutorial/markov-chain-analysis-r

## Input data created by the original codes producing descriptive statistics of the population

**Data Tables Explanation**
1. Descriptive Data Table: **[IDI_Sandpit].[DL-MAA2023-41].[202306_output1_descriptives]**
This table provides a snapshot of DSS clients for the year 2021, focusing on their demographic profiles and current status within DSS services. Each record includes:

*   snz_uid: Unique identifier for each client.
*   age_group: Age category of the client.
*   need_band_number and need_band_desc: Numerical and descriptive
*   categorisations of the client's needs
*   gender: Client's gender.
*   disability_allowance: Indicates if the client receives a disability allowance (1 for yes, 0 for no).
*   education_level: The highest level of education attained.
*   income_band: Income category.
*   employment_status: Current employment status.
*   benefit_reception: Indicates if the client is receiving any additional benefits (Yes or No).

| snz_uid | age_group | need_band_number | need_band_desc | gender | disability_allowance | education_level | income_band | employment_status | benefit_reception |
|---------|-----------|------------------|----------------|--------|----------------------|-----------------|-------------|-------------------|-------------------|
| 001     | 19-44     | 5                | 05-19          | Male   | 1                    | University      | High        | Employed          | Yes               |
| 002     | 45-64     | 20               | 20-39          | Female | 0                    | High School     | Medium      | Unemployed        | Yes               |
| 003     | 3-18      | 0                | Zero           | Female | 0                    | None            | Low         | N/A               | No                |
| 004     | 65+       | 80               | 80-99          | Male   | 1                    | College         | Low         | Retired           | No                |
| 005     | 19-44     | 1                | 01-04          | Female | 1                    | High School     | Medium      | Employed          | No                |


2.  Service Engagement Data Table: **[IDI_Sandpit].[DL-MAA2023-41].[202306_sample_input]**
This table contains entries for client engagements with DSS services:

*   snz_uid: Unique identifier for each client linked to the descriptive data.
*   age, gender, ethnicity: Demographic details.
*   disability_type: Type of disability the client has.
*   service_type: The type of DSS service the client is receiving.
*   entry_date and exit_date: Dates marking the start and potential end of service engagement.
*   status: Current status of the service engagement (Active, Exited).

| snz_uid | age | gender | ethnicity | disability_type  | service_type       | entry_date | exit_date | status |
|---------|-----|--------|-----------|------------------|--------------------|------------|-----------|--------|
| 001     | 34  | M      | European  | Cerebral Palsy   | Home Care          | 2018-01-01 | NULL      | Active |
| 002     | 22  | F      | Maori     | Autism           | Community Support  | 2017-06-01 | 2019-03-01| Exited |


## Data Preparation for Markov Chain Modelling
To prepare the data for Markov Chain Modelling, we need to consolidate and reformat the existing datasets to capture transitions between states (services) over time. This involves creating a new dataset that tracks changes in service status, type, and personal demographics year by year. The dataset will look like this:

Markov Chain Modelling Input Data Table
*   snz_uid: Unique identifier for each client.
*   year: The year of the record.
*   age: Client's age at the time of the record.
*   gender: Client's gender.
*   ethnicity: Client's ethnicity.
*   service_type: Type of service the client was receiving. However, in Markov Chain Modelling, these states are mutually exclusive and exhaustive and so each individual represented in the model can be in one and only one of these disease states at any given time.
*   entry_date and exit_date: Dates of service engagement.
*   reason_for_exit: Reason for service discontinuation (if applicable).
*   status: Status of the client in the DSS system (Active, Exited).

| snz_uid | year | age | gender | ethnicity | service_type        | entry_date  | exit_date  | reason_for_exit     | status  |
|---------|------|-----|--------|-----------|---------------------|-------------|------------|---------------------|---------|
| 001     | 2007 | 34  | M      | European  | Home Care           | 2007-01-05  | 2009-12-15 | Graduated           | Exited  |
| 002     | 2008 | 22  | F      | Maori     | Community Support   | 2008-06-10  | NULL       | NULL                | Active  |
| 003     | 2007 | 29  | M      | Pacific   | Job Training        | 2007-02-20  | 2008-03-01 | Job Found           | Exited  |
| 004     | 2009 | 18  | F      | Asian     | Education           | 2009-05-15  | 2013-06-30 | Graduated           | Exited  |
| 005     | 2010 | 45  | F      | European  | Rehabilitation      | 2010-09-01  | 2012-11-20 | Health Improvement  | Exited  |


## Initial Assumptions

### Population
This is a cohort model, i.e. it represents the experience of a simulated cohort, i.e. clients who receive (or do not receive) DSS in a certain perido of time.

### States in the Markov Model
To effectively model transitions between the states using a Markov Chain, we first need to clearly define each state, establish the conditions for transitions between them, and then collect the appropriate data to calculate the transition probabilities. Here's how we might set up your model with these states:

### Different DSS Services:

1. Home
*   Flexible funding
*   Facility-based support
*   Early investment
2. Employment Status (categorised by plausible and meaningful income bands):
*   No income
*   0-50% of minimum wage
*   50-100% of minimum wage
*   100% of minimum wage to average wage
*   Above average wage
3. Main Benefits:
*   Job seeker
*   Parental
*   Sickness
4. Supplementary Benefit
5. In Education
6. In Healthcare Facility (hospitalized)
7. In Prison
8. Deceased (Terminal state with no outgoing transitions)
9. Overseas
10. None of the Above




In [2]:
# Load necessary libraries
library(dplyr)
library(lubridate)

# Expanded example data frame
data <- data.frame(
  snz_uid = c("001", "002", "003", "004", "005", "006", "007", "008", "009", "010", "011", "012", "013", "014", "015"),
  year = c(2007, 2008, 2007, 2009, 2010, 2007, 2008, 2007, 2009, 2010, 2007, 2008, 2007, 2009, 2010),
  service_type = c("Home Care", "Community Support", "Job Training", "Education", "Rehabilitation",
                   "Home Care", "Community Support", "Job Training", "Education", "Rehabilitation",
                   "Home Care", "Community Support", "Job Training", "Education", "Rehabilitation"),
  entry_date = c("2007-01-05", "2008-06-10", "2007-02-20", "2009-05-15", "2010-09-01",
                 "2007-03-05", "2008-07-10", "2007-04-20", "2009-06-15", "2010-10-01",
                 "2007-02-01", "2008-08-10", "2007-05-20", "2009-07-15", "2010-11-01"),
  exit_date = c("2009-12-15", NA, "2008-03-01", "2013-06-30", "2012-11-20",
                "2009-11-15", NA, "2008-02-01", "2013-05-30", "2012-10-20",
                "2009-10-15", NA, "2008-01-01", "2013-04-30", "2012-09-20"),
  status = c("Exited", "Active", "Exited", "Exited", "Exited",
             "Exited", "Active", "Exited", "Exited", "Exited",
             "Exited", "Active", "Exited", "Exited", "Exited"),
  ethnicity = c("European", "Maori", "Pacific", "Asian", "European",
                "Maori", "Pacific", "European", "Asian", "Maori",
                "Pacific", "European", "Asian", "Maori", "Pacific")
)

# Convert date columns to Date type
data$entry_date <- as.Date(data$entry_date)
data$exit_date <- as.Date(data$exit_date, tryFormats = c("%Y-%m-%d", NA))

# Define transitions based on service type and status
data <- data %>%
  mutate(
    next_service = lead(service_type, default = "End of Data"),
    transition = paste(service_type, next_service, sep = " to ")
  )

# Display the dataset to see the new columns
print(data)




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




   snz_uid year      service_type entry_date  exit_date status ethnicity
1      001 2007         Home Care 2007-01-05 2009-12-15 Exited  European
2      002 2008 Community Support 2008-06-10       <NA> Active     Maori
3      003 2007      Job Training 2007-02-20 2008-03-01 Exited   Pacific
4      004 2009         Education 2009-05-15 2013-06-30 Exited     Asian
5      005 2010    Rehabilitation 2010-09-01 2012-11-20 Exited  European
6      006 2007         Home Care 2007-03-05 2009-11-15 Exited     Maori
7      007 2008 Community Support 2008-07-10       <NA> Active   Pacific
8      008 2007      Job Training 2007-04-20 2008-02-01 Exited  European
9      009 2009         Education 2009-06-15 2013-05-30 Exited     Asian
10     010 2010    Rehabilitation 2010-10-01 2012-10-20 Exited     Maori
11     011 2007         Home Care 2007-02-01 2009-10-15 Exited   Pacific
12     012 2008 Community Support 2008-08-10       <NA> Active  European
13     013 2007      Job Training 2007-05-20 2008-0

In [3]:
# Create a transition matrix from this data
transition_matrix <- table(data$service_type, data$next_service)

# Print transition matrix with more digits
print(format(transition_matrix, digits = 4))

# Note: You should replace "lead(service_type, default = "End of Data")" with appropriate logic
# to match your research requirements and data structure.

# This is just a basic example. Depending on your requirements, you may need to
# include more sophisticated data manipulation and analysis steps.


                   
                    Community Support Education End of Data Home Care
  Community Support "0"               "0"       "0"         "0"      
  Education         "0"               "0"       "0"         "0"      
  Home Care         "3"               "0"       "0"         "0"      
  Job Training      "0"               "3"       "0"         "0"      
  Rehabilitation    "0"               "0"       "1"         "2"      
                   
                    Job Training Rehabilitation
  Community Support "3"          "0"           
  Education         "0"          "3"           
  Home Care         "0"          "0"           
  Job Training      "0"          "0"           
  Rehabilitation    "0"          "0"           


Based on the matrix provided:

Community Support to Job Training: 3 transitions
Education to Rehabilitation: 3 transitions
Rehabilitation to Home Care: 2 transitions
Rehabilitation to End of Data: 1 transition


In [5]:
# Assuming data has columns 'service_type' and 'next_service'
# Create a transition matrix from this data
transition_matrix <- table(data$service_type, data$next_service)

# Add row totals to the transition matrix
transition_matrix <- cbind(transition_matrix, rowSums(transition_matrix))
colnames(transition_matrix)[ncol(transition_matrix)] <- "Total Out"

# Add column totals to the transition matrix
transition_matrix <- rbind(transition_matrix, colSums(transition_matrix))
rownames(transition_matrix)[nrow(transition_matrix)] <- "Total In"

# Calculate transition probabilities as percentages
transition_probabilities <- transition_matrix[1:(nrow(transition_matrix) - 1), 1:(ncol(transition_matrix) - 1)] /
                            rowSums(transition_matrix[1:(nrow(transition_matrix) - 1), 1:(ncol(transition_matrix) - 1)])

# Replace NA values with zeros (where division by zero occurs)
transition_probabilities[is.na(transition_probabilities)] <- 0

# Convert to percentages for clarity
transition_probabilities <- round(transition_probabilities * 100, 2)

# Print matrices
print("Transition Counts (with totals):")
print(transition_matrix)

print("Transition Probabilities (%):")
print(transition_probabilities)


[1] "Transition Counts (with totals):"
                  Community Support Education End of Data Home Care
Community Support                 0         0           0         0
Education                         0         0           0         0
Home Care                         3         0           0         0
Job Training                      0         3           0         0
Rehabilitation                    0         0           1         2
Total In                          3         3           1         2
                  Job Training Rehabilitation Total Out
Community Support            3              0         3
Education                    0              3         3
Home Care                    0              0         3
Job Training                 0              0         3
Rehabilitation               0              0         3
Total In                     3              3        15
[1] "Transition Probabilities (%):"
                  Community Support Education End of Data

Transition Probabilities (%):
This matrix converts the absolute counts into probabilities (or percentages) that help understand the likelihood of transitioning from one state to another.

Values represent the percentage probability of moving from the row state to the column state.
For example, 100% of individuals in Home Care transitioned away from Home Care, with none staying or moving to other states listed.
In Rehabilitation, 33.33% led to an "End of Data" state, and 66.67% transitioned back to Home Care, indicating a high likelihood of returning to initial care after Rehabilitation.

## Model Initialisation and Simulation:
Initiate the Markov Model: Start the Markov chain model using the initial state distribution based on the most recent data available. This involves setting up the initial probabilities of each state based on the data.
Run Simulations: Use the transition probabilities to simulate the movement of individuals through different states over multiple time periods. This simulation can extend several years into the future to observe long-term trends and potential impacts of current policies.

In [6]:
# Initialise the Markov Model


# Create a transition matrix from this data
transition_matrix <- table(data$service_type, data$next_service)

# Add row totals to the transition matrix for outgoing states
transition_matrix <- cbind(transition_matrix, rowSums(transition_matrix))
colnames(transition_matrix)[ncol(transition_matrix)] <- "Total Out"

# Add column totals to the transition matrix for incoming states
transition_matrix <- rbind(transition_matrix, colSums(transition_matrix))
rownames(transition_matrix)[nrow(transition_matrix)] <- "Total In"

# Calculate transition probabilities as percentages
transition_probabilities <- sweep(transition_matrix[1:nrow(transition_matrix) - 1, 1:ncol(transition_matrix) - 1], 1,
                                  rowSums(transition_matrix[1:nrow(transition_matrix) - 1, 1:ncol(transition_matrix) - 1]), "/") * 100
transition_probabilities[is.na(transition_probabilities)] <- 0  # Replace NA with zero

# Print transition matrices
print("Transition Counts (with totals):")
print(transition_matrix)

print("Transition Probabilities (%):")
print(transition_probabilities)



[1] "Transition Counts (with totals):"
                  Community Support Education End of Data Home Care
Community Support                 0         0           0         0
Education                         0         0           0         0
Home Care                         3         0           0         0
Job Training                      0         3           0         0
Rehabilitation                    0         0           1         2
Total In                          3         3           1         2
                  Job Training Rehabilitation Total Out
Community Support            3              0         3
Education                    0              3         3
Home Care                    0              0         3
Job Training                 0              0         3
Rehabilitation               0              0         3
Total In                     3              3        15
[1] "Transition Probabilities (%):"
                  Community Support Education End of Data

### Simulate the Markov Chain

In [10]:
# Install the markovchain package if it's not already installed
if (!require(markovchain)) {
  install.packages("markovchain", dependencies = TRUE)
}

# Load the package
library(markovchain)


Loading required package: markovchain

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘markovchain’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘htmlwidgets’, ‘visNetwork’, ‘mvtnorm’, ‘truncnorm’, ‘coda’, ‘numDeriv’, ‘igraph’, ‘expm’, ‘Rcpp’, ‘RcppParallel’, ‘RcppArmadillo’, ‘diagram’, ‘DiagrammeR’, ‘msm’, ‘Rsolnp’, ‘ctmcd’, ‘bookdown’, ‘rticles’


Package:  markovchain
Version:  0.9.5
Date:     2023-09-24 09:20:02 UTC
BugReport: https://github.com/spedygiorgio/markovchain/issues



Attaching package: ‘markovchain’


The following object is masked from ‘package:lubridate’:

    period




In [12]:
class(transition_matrix)

[1] "matrix" "array"

In [14]:
states <- unique(data$service_type)

print(states)

[1] "Home Care"         "Community Support" "Job Training"     
[4] "Education"         "Rehabilitation"   


In [11]:

# Define state names
states <- unique(data$service_type)
# Create the Markov chain object
mc <- new("markovchain", states = states,
          byrow = TRUE, transitionMatrix = transition_matrix,
          name = "DSS Service Transition")

# Print the Markov chain
print(mc)


ERROR: Error in dimnames(x) <- dn: length of 'dimnames' [2] not equal to array extent
